<a href="https://colab.research.google.com/github/sanjeekan/fruit-detection1/blob/main/fruit_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [3]:
dataset_path = "/content/drive/MyDrive/Fruits Classification"
categories = ['Apple', 'Banana', 'Grape', 'Mango', 'Strawberry']

img_size = 128
def load_images_and_labels(folder_path, categories):
  data = []
  labels = []
  for label, category in enumerate(categories):
    category_path = os.path.join(folder_path, category)
    for img_name in os.listdir(category_path):
      img_path = os.path.join(category_path, img_name)
      img = cv2.imread(img_path)
      img = cv2.resize(img, (img_size, img_size))
      data.append(img)
      labels.append(label)
  return np.array(data, dtype = 'float32') / 255.0, np.array(labels)

#load train, test, validation data
x_train, y_train = load_images_and_labels(os.path.join(dataset_path, "train"), categories)
x_valid, y_valid = load_images_and_labels(os.path.join(dataset_path, "valid"), categories)
x_test, y_test = load_images_and_labels(os.path.join(dataset_path, "test"), categories)


In [7]:
#Extract features
sift = cv2.SIFT_create()

def extract_sift_features(img):
  img = (img * 255).astype(np.uint8)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  keypoints, descriptors = sift.detectAndCompute(gray, None)
  if descriptors is None:
    return np.zeros(128)
  return np.mean(descriptors, axis = 0)

x_train_features = np.array([extract_sift_features(img) for img in x_train])
x_valid_features = np.array([extract_sift_features(img) for img in x_valid])
x_test_features = np.array([extract_sift_features(img) for img in x_test])

In [9]:
#train the classifier
svm = SVC(kernel = 'linear', random_state = 42)
svm.fit(x_train_features, y_train)

y_valid_pred = svm.predict(x_valid_features)
print("Validation Accuracy:", accuracy_score(y_valid, y_valid_pred))
print(classification_report(y_valid, y_valid_pred, target_names=categories))

Validation Accuracy: 0.45
              precision    recall  f1-score   support

       Apple       0.48      0.40      0.44        40
      Banana       0.51      0.55      0.53        40
       Grape       0.39      0.55      0.45        40
       Mango       0.45      0.35      0.39        40
  Strawberry       0.44      0.40      0.42        40

    accuracy                           0.45       200
   macro avg       0.46      0.45      0.45       200
weighted avg       0.46      0.45      0.45       200



In [10]:
#evaluate the performance of the model in test set
y_test_pred = svm.predict(x_test_features)
print("test_accuracy", accuracy_score(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred, target_names = categories))

test_accuracy 0.54
              precision    recall  f1-score   support

       Apple       0.50      0.40      0.44        20
      Banana       0.57      0.65      0.60        20
       Grape       0.67      0.70      0.68        20
       Mango       0.48      0.60      0.53        20
  Strawberry       0.47      0.35      0.40        20

    accuracy                           0.54       100
   macro avg       0.54      0.54      0.53       100
weighted avg       0.54      0.54      0.53       100



In [12]:
def predict_image(image_path):
  img = cv2.imread(image_path)
  img_resized = cv2.resize(img, (img_size, img_size))
  features = extract_sift_features(img_resized)
  prediction = svm.predict([features])
  return categories[prediction[0]]

sample_image_path = "/content/drive/MyDrive/b1.jpg"
predicted_class = predict_image(sample_image_path)
print(f"predicted_class: {predicted_class}")


predicted_class: Apple


In [13]:
sample_image_path2 = "/content/drive/MyDrive/m1.jpg"
predicted_class = predict_image(sample_image_path2)
print(f"predicted_class: {predicted_class}")


predicted_class: Mango


In [14]:
sample_image_path3 = "/content/drive/MyDrive/g1.jpg"
predicted_class = predict_image(sample_image_path3)
print(f"predicted_class: {predicted_class}")


predicted_class: Strawberry


In [15]:
sample_image_path4 = "/content/drive/MyDrive/s1.jpg"
predicted_class = predict_image(sample_image_path4)
print(f"predicted_class: {predicted_class}")


predicted_class: Mango
